In [3]:
import math
import numpy as np
from matplotlib import pyplot as plot
import random
import time
import pandas as pd

In [4]:
def f_of_x5(x):
    s=0
    for i in range(len(x)):
        s+=(x[i]-i**2)**2
    return s

def f_of_x1(x):
    # Rosenbrock function
    s=0
    for i in range(len(x)-1):
        s+=100*(x[i+1]-(x[i]**2))**2+(x[i]-1)**2
    return s

def f_of_x2(x):
    # Beale function
    if len(x)!=2:
        print('invalid length')
        return            
    return ((1.5-x[0]+x[0]*x[1])**2)+((2.25-x[0]+x[0]*(x[1]**2))**2)+((2.625-x[0]+x[0]*(x[1]**3))**2)

def f_of_x3(x):
    # Easom function
    if len(x)!=2:
        print('invalid length')
        return            
    return -math.cos(x[0])*math.cos(x[1])*math.exp(-(((x[0]-math.pi)**2)+((x[1]-math.pi)**2)))

def f_of_x4(x):
    # Booth function
    if len(x)!=2:
        print('invalid length')
        return            
    return ((x[0]+2*x[1]-7)**2)+((2*x[0]+x[1]-5)**2)

def f_of_x6(x):
    # Eggholder function
    if len(x)!=2:
        print('invalid length')
        return            
    return -(x[1]+47)*math.sin(math.sqrt(np.abs(x[0]/2+(x[1]+47)))) - x[0]*math.sin(math.sqrt(np.abs(x[0]-(x[1]+47))))

def f_val(x, f_of_x):

    M,npar = np.shape(x)
    if npar==1:
        f = []
    else:
        f = np.empty(shape=(M,),dtype='float')
    for i in range(M):
        if npar>1:
            f[i] = f_of_x(x[i][:])
        else:
            f.append(f_of_x(x[i][:]))
    return f

In [5]:
def decode(ch,bits,hi,lo):
    quant = np.empty(shape=(bits,),dtype='float')
    
    if type(ch)==np.ndarray:
        M,N = np.shape(ch)
        npar = int(N/bits)
        ct1 = np.empty(shape=(M,npar),dtype='float')
    for i in range(bits):
        quant[i] = 0.5**(i)
    quant = (quant/sum(quant))
    ct = ch.reshape(npar*M,bits)
    ct1 = ((np.dot(ct,quant)).reshape(M,npar))
    if npar==1:
        
        par = (ct1*(hi-lo)+lo).reshape(M,)
    else:
        par = np.empty(shape= (M,npar))
        for i in range(M):
            for j in range(npar):
                par[i][j] = ct1[i][j]*(hi[j]-lo[j])+lo[j]
    return par

In [6]:
def init_optimization(f_num):
    if f_num==1:
        hi = 5
        lo = -2
        dim = 3
        f = f_of_x1
        print('Rosenbrock Function in 4 dimension')
        print('Global minimum f(x_i) = 0 for x_i in [-inf,inf]')
    elif f_num==2:
        hi = 4.5
        lo = -4.5
        dim = 2
        f = f_of_x2
        print('Beale function in 2 dimension')
        print('Global minimum f(3,.5) = 0 for x,y in [-5,5]')
    elif f_num==3:
        hi = 10
        lo = -10
        dim = 2
        f = f_of_x3
        print('Eason Function in 2 dimension')
        print('Global minimum f(pi,pi) = -1 for x,y in [-10,10]')
    elif f_num==4:
        hi = 10
        lo = -10
        dim = 2
        f = f_of_x4
        print('Booth Function in 2 dimension')
        print('Global minimum f(1,3) = 0 for x,y in [-10,10]')
    elif f_num==5:
        hi = 50
        lo = -2
        dim = 6
        f = f_of_x5
        print('generated Function in 5 dimension')
        print('Global minimum f(i^2) = 0 for x_i in [-2,50]')
    elif f_num==6:
        hi = 512
        lo = -512
        dim = 2
        f = f_of_x6
        print('Eggholder function in 2 dimension')
        print('Global minimum f(512,404.2319) = -959.6407 for x_i in [-512,512]')
    else:
        print('provide proper value for function to be optimized')
    return hi,lo,dim, f

In [9]:
import time;

localtime = time.asctime( time.localtime(time.time()) )

print('_________________________________________')
print('_____Genetic algorithm modified_______')
print('_________________________________________')
print('Local current time :', localtime)
for opti in range(1,6):
    tic = time.time()
    h,l,dim, f= init_optimization(opti)
    ff=f # objective function
    npar=dim # number of optimization variables
    hi=np.ones((dim,), dtype=int)*h
    lo=np.ones((dim,), dtype=int)*l
    maxit=5000 # max number of iterations
    mincost=-99999 # minimum cost
    #_______________________________________________________
    # III. GA parameters
    popsize=64# set population size
    mutrate=.15# set mutation rate
    selection=0.2 # fraction of population kept
    nbits=16# n4umber of bits in each parameter
    Nt=nbits*npar # total number of bits
    tol = 10**-8
    # in a chormosome
    keep=math.floor(selection*popsize) # #population members
    # that survive
    #_______________________________________________________
    # Create the initial population
    itr=0 # generation counter
    # initialized
    pop=np.round(np.random.rand(popsize,Nt)) # random population of 1s and 0s
    par=decode(pop,nbits,hi,lo) # convert binary to continuous values

    cost=f_val(par,ff) # calculates population
    # cost using ff
    ind=np.argsort(cost,axis=0) # min cost in element 1
    cost=cost[ind]

    par = par[:][ind]
    pop=pop[:][ind] # sorts population with
    # lowest cost first
    minc = []
    meanc = []
    minc.append(np.min(cost)) # minc contains min of
    # population
    meanc.append(np.mean(cost)) # meanc contains mean
    # of population
    while itr < maxit+1 and np.abs(cost[0])> tol:
        # Performs mating using single point crossover
        M = math.ceil((popsize-keep)/2)

        xp=np.random.rand(M)*(Nt-1) # crossover point
        ma=np.empty(shape=(M,Nt))
        pa=np.empty(shape=(M,Nt))

        for k in range(M):
            ma[k]=pop[2*k][:]
            pa[k]=pop[2*k+1][:]

            pop[keep+2*k][:]=np.append(ma[k,0:math.ceil(xp[k])],pa[k,math.ceil(xp[k]):Nt],axis=0)
            # first offspring
            pop[keep+2*k+1][:]=np.append(pa[k,0:math.ceil(xp[k])],ma[k,math.ceil(xp[k]):Nt],axis=0)
            # second offspring
        #_______________________________________________________
        # Mutate the population4
        nmut=math.ceil((popsize-1)*Nt*mutrate)-1 # total number
        # of mutations
        mrow=np.random.rand(nmut)*(popsize-1) # row to mutate
        mcol=np.random.rand(nmut)*(Nt-1) # column to mutate

        mr=[]
        mc=[]
        for ii in range(nmut):
            mr.append(math.ceil(mrow[ii]))
            mc.append(math.ceil(mcol[ii]))

        for ii in range(nmut):
            pop[mr[ii]][mc[ii]]=np.absolute(pop[mr[ii]][mc[ii]]-1)
         # ii
        #_______________________________________________________
        # The population is re-evaluated for cost
        par=decode(pop,nbits,hi,lo)
        # decode
        cost=f_val(par,ff)#_val(par,ff,popsize,npar)
        #_______________________________________________________
        # Sort the costs and associated parameters
        ind=np.argsort(cost,axis=0) # min cost in element 1
        cost = cost[ind]
        #    print('sorted cost',cost)
        par=par[:][ind]
        #    print('sorted p',par)
        pop=pop[:][ind]
        #_______________________________________________________
        # Do statistics for a single nonaveraging run
        minc.append(np.min(cost))
        meanc.append(np.mean(cost))
        #_______________________________________________________
        # Stopping criteria
        if itr>maxit or np.abs(cost[0])<tol : # or np.absolute(minc[itr+1]-minc[itr])<tol:
            break

        if itr%5000==0:
            print('iteration : ', str(itr), ', cost : ',str(cost[0]))
        
        itr +=1
    toc = time.time()
    print('-----------------------------------')
    print('final output :')
    print(par[0][:],itr,cost[0])
    print('elapsed time',toc-tic)
    print('____________________________________')

_________________________________________
_____Genetic algorithm modified_______
_________________________________________
Local current time : Fri Feb  5 20:43:01 2021
Rosenbrock Function in 4 dimension
Global minimum f(x_i) = 0 for x_i in [-inf,inf]
iteration :  0 , cost :  15.474479875299195
iteration :  5000 , cost :  2.1854763384915632e-05
-----------------------------------
final output :
[1.00209049 1.00411994 1.00828565] 5001 2.1854763384915632e-05
elapsed time 10.639063835144043
____________________________________
Beale function in 2 dimension
Global minimum f(3,.5) = 0 for x,y in [-5,5]
iteration :  0 , cost :  0.8156312825692068
-----------------------------------
final output :
[2.999794   0.49995422] 1018 7.443245336512953e-09
elapsed time 1.588759183883667
____________________________________
Eason Function in 2 dimension
Global minimum f(pi,pi) = -1 for x,y in [-10,10]
iteration :  0 , cost :  -0.01511270249798119
iteration :  5000 , cost :  -0.9999999771022167
--------